In [1]:
import time
import math
import json
import pandas as pd
from datetime import datetime
import telebot
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
from binance.enums import *
import urllib3

urllib3.disable_warnings()

In [2]:
with open('binance.json') as f:
    api_keys = json.load(f)

In [3]:
def get_pair(spot_symbols, futures_symbols):
    pair = []
    for spot_symbol in spot_symbols:
        try:
            temp_symbols = []
            for future_symbol in futures_symbols:
                if spot_symbol in future_symbol: 
                    temp_symbols.append(future_symbol)

            futures_symbol = temp_symbols[0]
            pair.append({"spot":spot_symbol, "futures":futures_symbol})
        except IndexError: 
            pass
    return pair

In [4]:
def check_bid_ask(client, pair):
    s_quotes = client.get_orderbook_ticker()
    df_s = pd.DataFrame(s_quotes)
    f_quotes = client.futures_orderbook_ticker()
    df_f = pd.DataFrame(f_quotes)
    df_merge = df_s.merge(df_f, how='inner', on='symbol', suffixes=('_s', '_f'))
    df_merge = df_merge.astype(
        {
            "symbol": str,
            "askPrice_s": 'float64',
            "askPrice_f": 'float64',
            "bidPrice_s": 'float64',
            "bidPrice_f": 'float64',
            "askQty_s": 'float64',
            "askQty_f": 'float64',
            "bidQty_s": 'float64',
            "bidQty_f": 'float64'
        }
    )
    
    df_merge['diff_s'] = ((df_merge['bidPrice_s'] - df_merge['askPrice_f'])*100)/df_merge['bidPrice_s']
    df_merge['diff_f'] = ((df_merge['bidPrice_f'] - df_merge['askPrice_s'])*100)/df_merge['bidPrice_f']
    df_merge = df_merge.drop({"time", "bidQty_s", "askQty_s", "bidQty_f", "askQty_f"}, axis=1)
    return df_merge

In [5]:
def update_diff(bot, diff_df, symbol, diff_s, diff_f):
    id = diff_df[diff_df['symbol']==symbol].index.values[0]

    if diff_df.iloc[id]['diff_s'] < diff_s:
        diff_df.loc[diff_df["symbol"]==symbol,'diff_s'] = diff_s
        msg = symbol+": short spot "+str(round(diff_s, 4))
        print(msg)
        bot.send_message(CHANEL_ID, msg)
    
    if diff_df.iloc[id]['diff_f'] < diff_f:
        diff_df.loc[diff_df["symbol"]==symbol,'diff_f'] = diff_f
        msg = symbol+": short futures "+str(round(diff_f, 4))
        print(msg)
        bot.send_message(CHANEL_ID, msg)

In [6]:
#Создаем binance клиента
client = Client(api_keys['apiKey'], api_keys['secret'], {"verify": False, "timeout": 20})

#Формируем списки спота и фьючерса
spot_info = client.get_all_tickers()
spot_symbols = [market['symbol'] for market in spot_info]

futures_info = client.futures_exchange_info()
futures_info_symbols = futures_info['symbols']
futures_symbols =  [market['symbol'] for market in futures_info_symbols]

#Создаем список повторяющихся пар спот-фьючерс
pair = get_pair(spot_symbols, futures_symbols)

#Создаем экземпляр телеграмм бота
BOT_KEY = api_keys["telebot_key"]
CHANEL_ID = api_keys["CHANEL_ID"]
bot = telebot.TeleBot(BOT_KEY)

In [7]:
#Спрашиваем котровки бидов и асков на спол и фьючерс. Сравниваем и выдаем разницу
MIN_DIFF = 3
PAUSE_SEC = 3
data = []
for sym in pair:
    data.append({'symbol':sym["spot"], 'diff_s':0.00, 'diff_f':0.00})
    
diff_df = pd.DataFrame(data)

while(1):
    try:
        df = check_bid_ask(client, pair)
        spot_mask = df.loc[df['diff_s']>MIN_DIFF].index
        futures_mask = df.loc[df['diff_f']>MIN_DIFF].index
        if not spot_mask.empty:
            for i in spot_mask:
                update_diff(bot, diff_df, df.loc[i].symbol, df.loc[i].diff_s, df.loc[i].diff_f)
        if not futures_mask.empty:
            for i in futures_short:
                update_diff(bot, diff_df, df.loc[i].symbol, df.loc[i].diff_s, df.loc[i].diff_f)
        time.sleep(PAUSE_SEC)
    except Exception as ex:
        print(ex)
        print("Binance Timeout err. Wait 1min")
        time.sleep(60)

KeyboardInterrupt: 